# Introduction: 
This lesson covers how to navigate the Raspberry Pi Pico/W, the capacitive soil moisture sensor, and MicroPython, and analyze the data collected. 

# Materials:

* Raspberry Pi Pico W
* Capacitive Soil Moisture Sensor v1.2
* Sparkfun Qwiic pHAT Extension
* Five male-to-female jumper wires
* One 330 Ohm resistor


## Other Materials:
* 8 cups
* 2(A-B)-4(A-D) different types of soil



*Note: Include Thonny Set Up???

# Wiring Diagram: ???

# MicroPython Implementation & Data Collection:

Copy and paste the code below into Thonny and upload into capacitive soil moisture sensor

In [1]:
from machine import Pin, ADC
import json

import utime
#
adc = ADC(Pin(28))
print(adc.read_u16())
#print(adc.read_uv())

class Moisture_Sensor:
    def __init__(self, mqtt_handle=None, pin_num=28, sensor_id="soil_moisture_1", topic="sensor/moisture", indicator_pin="LED"):
       utime.sleep_us(100)

       self.sensor_id = sensor_id
       self.topic = topic
       self.pin = ADC(Pin(pin_num))
       self._raw_value = 0
       self.calibration_value_low = 0
       self.calibration_value_high = 65535
       self.indicator_pin = Pin(indicator_pin, Pin.OUT)
       self.mqtt_handle = mqtt_handle

    def toggle_indicator(self):
        self.indicator_pin.toggle()

    def read_moisture(self):
        self.indicator_pin.on()
        self._raw_value = self.pin.read_u16()
        utime.sleep(1)
        self.indicator_pin.off()
        print(self._raw_value)
        if self.mqtt_handle is None:
            return self._raw_value
        else:
            return None
        # return self._raw_value.to_bytes(2, 'big')

    def publish(self):
        if self.mqtt_handle is not None:
            self.read_moisture()
            msg = json.dumps({self.sensor_id : (self._raw_value / self.calibration_value_high)})
            self.mqtt_handle.connect()
            self.mqtt_handle.publish(self.topic, msg)
            self.mqtt_handle.disconnect()
        else:
            raise AttributeError("No MQTT handler defined for this sensor")


if __name__ == '__main__':
    test_ms = Moisture_Sensor()

    while True:
        reading = test_ms.read_moisture()
        print(reading)
        utime.sleep(2)

ModuleNotFoundError: No module named 'machine'

# Lesson Instructions:
1. Copy and paste code into Thonny, and upload into soil moisture sensor
2. Gently insert soil sensor into soil into cup A. Make sure sensor is fully submeerged in soil.
3. Begin a timer or stopwatch to measure for two minutes. Allow sensor to collect data for two minutes and move onto next cup.
4. After collecting all cup measurements, examine and compare moisture measurements.
5. Identify which soil had the most soil moisture levels and record results.
6. Consider the implications of the moisture levels you've measured. For example, you can use this information to determine which type of soil retains the most moisture, which can be valuable for gardening or plant care.
7. Repeat as Necessary: If you have more soil cups or want to verify your results, repeat the measurement process for additional soil samples.

# Data Analysis:
Based on the data collected we can determine some factors that may have affected the soil moisture content.

**Question 1.** The dataset is in a file called `soil_data.csv`.  Load it into a table named `soil_data`.

1. What was the largest increase in moisture based on the data set?

In [ ]:
#Load CSV File
soil_data = Table.read_table("soil_data.csv").column("data")
soil_data.show(15)

In [ ]:
#prime loop: establishing the first conditional check in a loop
biggest_increase = 0

#Create a "for loop": used to iterate/repeat a specific/known amount of times
#enumerate funtion keeps track of the amount of iterations
for i, data in enumerate(soil_data):
    if i == 0:
        previous_num = data
    else:
        #prime
        change = data - previous_num
        if change > 0:
            if change > biggest_increase:
                biggest_increase = change
        #reprime loop
        previous_num = data

#Print calculated increase
print("Biggest increase is: ", biggest_increase)

NameError: name 'soil_data' is not defined

**Question 2.** 

2. From the dashboard, compare your graph to others that used different types of soil. Can you make any inferences based on the data compared?